In [1]:
import copy
steps = []  

def is_acyclic(graph):
    return graph.is_forest()

def feedback_vertex_set(graph, k, visited=None):
    global steps
    if visited is None:
        visited = set()
        steps = [(deepcopy(graph), k, "Original")]  


    if is_acyclic(graph) and k >= 0:
        steps.append((deepcopy(graph), k, "Graph is acyclic, solution found"))
        return True
    
    if k < 0:
        return False

    for v in graph.vertices():
        # Loop
        if graph.has_edge(v, v):
            graph.delete_vertex(v)
            steps.append((deepcopy(graph), k, f"Removed loop at {v}"))
            if feedback_vertex_set(graph, k - 1, visited):
                return True

    for u, v in graph.edges(labels=False):
        # Multiedge
        if graph.multiple_edges(u, v):
            graph.merge_edges([(u, v)], loops=True)
            steps.append((deepcopy(graph), k, f"Merged multiple edges between {u} and {v}"))
    
    for v in graph.vertices():
        # Degree-1
        if graph.degree(v) == 1:
            graph_copy = deepcopy(graph)
            graph_copy.delete_vertex(v)
            steps.append((graph_copy, k, f"Removed degree-1 vertex {v}"))
            return feedback_vertex_set(graph_copy, k, visited)
        
        # Degree-2
        elif graph.degree(v) == 2:
            u, w = graph.neighbors(v)
            graph_copy = deepcopy(graph)
            graph_copy.delete_vertex(v)
            if not graph.has_edge(u, w):
                graph_copy.add_edge(u, w)
                if k >= 1:
                    steps.append((graph_copy, k - 1, f"Removed degree-2 vertex {v} and added edge {u}-{w}"))
            else:
                if k >= 1:
                    steps.append((graph_copy, k - 1, f"Removed degree-2 vertex {v} "))
            if feedback_vertex_set(graph_copy, k - 1, visited):
                return True
            
    # Recursive case
    for v in graph.vertices():
        if v not in visited:  # Avoid re-visiting the same vertex
            visited.add(v)
            graph_copy = deepcopy(graph)
            graph_copy.delete_vertex(v)
            if k >= 1:
                steps.append((deepcopy(graph_copy), k - 1, f"Trying removal of {v}"))
                if feedback_vertex_set(graph_copy, k - 1, visited):
                    return True

    return False

def main(graph, k, visited=None):
    result = feedback_vertex_set(graph, k, visited=None)
    if not result:
        steps.append((deepcopy(graph), k, f"No solution!"))
    return result


In [2]:
G = Graph([(1, 2), (2, 3), (3, 1), (3, 4), (4, 5), (5, 6), (6, 4)])  # Example graph
k = 1
print(main(G, k))

False


In [3]:
@interact
def show_step(step=slider(0, len(steps)-1, 1, label="Step")):
    graph, k, description = steps[step]
    G_plot = graph.plot(title=f"Step {step}: k={k}\n{description}",layout='circular')
    show(G_plot)

Interactive function <function show_step at 0x13ec00c10> with 1 widget
  step: TransformIntSlider(value=0, des…